# COMS W2132 Intermediate Computing in Python
## Linked Lists

**Date**: February 16, 2026\
Daniel Bauer (original notes by Jan Janak)

**Reading**: Data Structures and Algorithms in Python, Chapter 7

---

The built-in Python list data structure, which is based on arrays, has shortcomings that make it unsuitable for some applications:
  1. Inserting or removing elements in the middle or beginning of the list can be expensive (why?)
  1. The size of the array storing the elements could be twice the size of the elements in the worst-case (why?)
  1. Amortized $O(1)$ running times for append may be unacceptable for some applications (which?)

A **linked list** is an alternative to the built-in list data structure with the following key characteristics:
  * It does NOT use arrays (contiguous memory buffers of fixed size) to store elements
  * It does NOT support efficient ($O(1)$) indexing (consequence of not using arrays)
  * But adding/removing at either end is more efficient! ($O(1)$, and not amortized)

In other words, linked lists achieve better asymptotic running times of add/remove operations by doing away with arrays and giving up $O(1)$ indexing. They also consume more memory than array lists. The idea behind linked lists is embarrassingly simple:

<img src="https://janakj.org/w3132/images/linked-list.png"/>

We store each element in its memory location (in arbitrary order, and not necessarily in a contiguous block). Each element gets a bit of extra space (overhead). The extra space contains a "link" (memory pointer) to the next element in the sequence there. These links are where linked lists get their name from. _Having elements stored in arbitrary memory locations means we can no longer perform constant-time indexing._

## Singly Linked Lists

This is the simplest variant. A singly linked list is a collection of elements (Python objects) that form a linear sequence (logitically, not necessarily physically in memory). The elements are called **nodes**. Each node contains a reference to the Python object (the element) and a reference to the next node of the list.

In [ ]:
lst = ["Bravo", "Charlie", "Delta"]

<img src="https://janakj.org/w3132/images/nato-alphabet-list-1.png"/>

Terminology:
* **node**: A Python object encapsulating an element and a reference to the next node
* **element**: An arbitrary Python object, an element of the sequence
* **head**: The first node of the linked list
* **tail**: The last node of the linked list (has no reference to the next node)

### Finding an Element (Traversal)

Accessing list elements (a.k.a. traversing or link-hopping):
  1. Start at the head
  2. Follow the link at each node to get to the next node
  3. If the link is None, we have reached the end (tail)

### Inserting an Element at the Head

<img src="https://janakj.org/w3132/images/nato-alphabet-list-2.png"/>

  1. Create a new node object
  1. Set the new node's next to the list's head
  1. Set the list's head to the new node

Question: what is the asymptotic running time of this operation?

### Removing an Element from the Head

<img src="https://janakj.org/w3132/images/nato-alphabet-list-4.png"/>

1. If lists's head is None, indicate an error
2. Set list's head to list's head next

### Implementing a Basic Singly Linked List
We define two new classes. The class Node represents linked list nodes. The class SinglyLinkedList represents the entire list.

What methods should the SinglyLinkedList class provide? What operations can we implement in constant time? They are:
  * Add a new element at the beginning (head) of the list
  * Remove an element from the head of the list

In [45]:
class Empty(Exception):
    pass


class Node:
    '''The Node class represents an element of a singly linked list

    The node contains two references: a reference to the value and
    a reference to the next node.
    '''
    def __init__(self, element, next=None):
        self.element = element   # A reference to the value
        self.next = next         # A reference to the next node


class SinglyLinkedList:
    'A basic singly linked list implementation'

    def __init__(self):
        self.head = None

    def add_first(self, element):
        'Add a new element at the head of the linked list'
        # 1. Create a node
        # 2. Set the node's next pointer to current head
        new_node = Node(element, self.head)
        # 3. Update the head pointer
        self.head = new_node

    def remove_first(self):
        'Remove and return the first element from the head of the list'

        # First check if we have any elements in the list. If not, raise
        # the Empty exception (defined earlier for stacks and queues)
        if self.head is None:
            raise Empty('The list is empty')

        # Save a reference to the current node at the head of the list
        node = self.head

        # Update the head pointer to point to the next element. If there
        # is no next element, it will be set to None
        self.head = self.head.next

        # Since node is no longer in the list, we can set its _next
        # reference to None to help the Python garbage collector
        node.next = None

        # Finally, return the node's value
        return node.element

### Keeping a Reference to the Tail

Sometimes, it is also useful to keep a reference to the tail of the singly linked list. Keeping a tail reference allows us to jump straight to the end. It also allows us to append new elements at the end of the list. We can extend our class SinglyLinkedList, adding the tail reference, and also the size of the list (will be useful later).

Having a reference to the tail, we can also implement constant-time `add_last()` method which will add the element to the end of the list.

<img src="https://janakj.org/w3132/images/nato-alphabet-list-3.png"/>

1. Create a new node object
1. Set the new node's next to None
1. Set the lists's tail next to the new node
1. Set the lists's tail to the new node
1. Update head
1. Increase the size

Question: What is the asymptotic running time of this operation?

Unfortunately, keeping the tail reference also complicates the implementation of `add_first()` and `remove_first()` methods, as they need to handle various corner cases when the list is empty and has the head and tail references set to None.

In [56]:
class SinglyLinkedList:
    'A basic singly linked list implementation with tail and size'

    def __init__(self):
        self.head = None
        self.tail = None  # NEW: We have added this instance variable
        self.size = 0     # NEW: We have added this instance variable
    
    def add_first(self, element):
        'Add a new element at the head of the linked list'
        # 1. Create a node
        # 2. Set the node's next pointer to current head
        new_node = Node(element, self.head)
        # 3. Update the head pointer
        self.head = new_node

        # If the list was previously empty, we also need to update the
        # tail to point to the newly added node. That is needed because
        # the list only contains one node, and both head and tail should
        # point to that node.
        if self.size == 0:
            self.tail = new_node

        self.size += 1
    
    def remove_first(self):
        'Remove and return the first element from the head of the list'

        # First check if we have any elements in the list. If not, raise
        # the Empty exception (defined earlier for stacks and queues)
        if self.size == 0:
            raise Empty('The list is empty')

        # Save a reference to the current node at the head of the list
        node = self.head

        # Update the head pointer to point to the next element. If there
        # is no next element, it will be set to None
        self.head = self.head.next

        # Since node is no longer in the list, we can set its _next
        # reference to None to help the Python garbage collector
        node.next = None

        self.size -= 1

        # If we removed the last element from the list, we also need to
        # set the tail pointer to None, since there are no more nodes
        if self.size == 0:
            self.tail = None
        
        # Finally, return the node's value
        return node.element

    # NEW: Keeping a reference to the tail allows us to implement the
    # following method in constant time
    def add_last(self, element):
        'Add a new element at the tail of the linked list'
        # 1. Create a new node
        new_node = Node(element, None)

        # 2. Set the previous node's next to the new node

        if self.size == 0:
            self.head = new_node
        else:    
            self.tail.next = new_node        
            # 3. Update the tail pointer

        self.tail = new_node
        self.size += 1

### Implementing a Stack using Singly Linked Lists

We can use our singly linked list implementation to implement a stack.

In [51]:
class LinkedStack:
    def __init__(self):
        self._data = SinglyLinkedList()

    def push(self, el):
        self._data.add_first(el)

    def pop(self):
        return self._data.remove_first()

In [52]:
stack = LinkedStack()
stack.push(42)
stack.push("alpha")
stack.push("python")

print(stack.pop()) # expecting: python
print(stack.pop()) # expecting: alpha
print(stack.pop()) # expecting: 42

python
alpha
42


#### Comparing Asymptotic Running Times

| Operation    | Linked List | Array (built-in Python List)  |
|--------------|-------------|------------------|
| S.push(e)    | $O(1)$      | amortized $O(1)$ |
| S.pop()      | $O(1)$      | amortized $O(1)$ |
| S.top()      | $O(1)$      | $O(1)$           |
| S.is_empty() | $O(1)$      | $O(1)$           |
| len(S)       | $O(1)$      | $O(1)$           |


### Implementing a Queue with a Singly Linked List

And we can also use the singly linked list to implement a queue:

In [53]:
class LinkedQueue:
    def __init__(self):
        self._data = SinglyLinkedList()
    
    def enque(self, el):
        self._data.add_last(el)

    def deque(self):
        return self._data.remove_first()

In [55]:
queue = LinkedQueue()
queue.enque(42)
queue.enque("alpha")
queue.enque("python")

print(queue.deque()) # expecting: 42
print(queue.deque()) # expecting: alpha
print(queue.deque()) # expecting: python

42
alpha
python


## Doubly Linked Lists

Singly-linked lists have some important shortcomings:
  * The list can be traversed from one direction only: from head to tail
  * We cannot easily delete the element at the end (tail) of the list
  * We cannot easily delete an element in the middle of the list (given a reference to it)

The limitations stem from the fact that a singly linked list maintains only one set of links: from one node to the next node. We need some mechanism to find the node that immediately precedes the given node.

### Headers and Trailers (Sentinels)

* The above singly linked list implementation needs to deal with special cases, such as when the list is empty
* This considerably complicates the implementation of the key methods
* The problem would get even worse in doubly linked lists, where we need to keep two sets of pointers

**Solution: Make sure the list is never empty using sentinel values!**

<img src="https://janakj.org/w3132/images/doubly-linked-list-sentinels.png"/>

Sentinels are fixed empty nodes before head and after tail. An empty doubly linked list consists of the two sentinel nodes linking to each other. The sentinels greatly simplify node operations. They guarantee that every node has a prev (even head) and that every node (even tail) has a next node.

* Sentinels greatly simplify the logic of operations
* Header and trailer never change (only nodes between them change)
* All insertions happen between existing nodes
* Removals are guaranteed to have nodes on both sides

### Extending the Node Class

We extend the node class developed for the singly linked list. The modification consists of adding the "prev" link.

In [79]:
class Node:
    def __init__(self, element, prev, next):
        self.element = element
        self.next = next
        self.prev = prev

We create a new class implementing an abstract doubly linked list. This version will not be ready for public consumption yet.

In [60]:
class DoublyLinkedList:
    def __init__(self):
        self.header = Node(None, None, None)
        self.trailer = Node(None, None, None)
        self.header.next = self.trailer
        self.trailer.prev = self.header
        self.size = 0

    #
    # Auxilialy methods
    #
    
    def __len__(self):
        pass

    def is_empty(self):
        pass

    def _insert_between(self, e, predecessor, successor):
        'Insert element e between the two nodes'
        pass
    
    def _delete_node(self, node):
        'Delete the node "node"'
        pass

### Inserting a Node

With the sentinel values, every insertion occurs between two existing nodes. We just need to update the links (in both directions) accordingly.

<img src="https://janakj.org/w3132/images/doubly-linked-list-insert.png"/>

In [81]:
def _insert_between(self, e, predecessor, successor):
    new_node = Node(e, predecessor, successor)
    predecessor.next = new_node
    successor.prev = new_node
    self.size += 1
    return new_node

### Removing a Node

Removing nodes from a doubly linked list is very similar. We will link the neighboring nodes directly to each other. Thanks to the sentinels, the neighboring nodes are guaranteed to exist.

<img src="https://janakj.org/w3132/images/doubly-linked-list-remove.png"/>

In [83]:
def _delete_node(self, node):
    el = node.element
    
    predecessor = node.prev
    successor = node.next

    successor.prev = node.prev
    predecessor.next = node.next
    self.size -= 1
 
    return el

### Performance Analysis

* The space used by each position in the list is $O(1)$
* The space used by a doubly linked list of $n$ elements is $O(n)$
* Adding/removing from either end run in worst-case $O(1)$ time.
* Indexing is still $O(n)$. But in many applications we can maintain a reference to a current node, and then navigate forward or backward.

_Interesting fact: Doubly-linked lists could support worst-case general (anywhere) $O(1)$ operations if, and only if, we already have a reference to the position within the list where the operation should be performed._

### Basic Doubly Linked List Implementation

In [85]:
# Not for public consumption yet
class DoublyLinkedList:
    def __init__(self):
        self.header = Node(None, None, None)
        self.trailer = Node(None, None, None)
        self.header.next = self.trailer
        self.trailer.prev = self.header
        self.size = 0

    #
    # Auxilialy methods
    #
    
    def __len__(self):
        return self.size

    def is_empty(self):
        return self.size == 0

    # Beware: you could mess this up
    def _insert_between(self, e, predecessor, successor):
        'Insert element e between the two nodes'
        new_node = Node(e, predecessor, successor)
        predecessor.next = new_node
        successor.prev = new_node
        self.size += 1
        return new_node

    # Beware: you could mess this up
    def _delete_node(self, node):
        'Delete the node "node"'
        el = node.element
        
        predecessor = node.prev
        successor = node.next
    
        successor.prev = node.prev
        predecessor.next = node.next
        self.size -= 1
     
        return el

### Example: Double-Ended Queue (Deque)

The deque is a generalization of the queue ADT we have seen earlier. The deque supports insertions and deletions at both ends of the queue. The Deque ADT looks as follows:

In [67]:
class Deque:
    def add_first(self, e):
        'Add element e to the front of the double-ended queue'
        pass

    def add_last(self, e):
        'Add element e to the end of the double-ended queue'
        pass

    # Renamed deque operation
    def remove_first(self):
        'Remove the first element from the double-ended queue'
        pass

    def remove_last(self):
        'Remove the last element from the double-ended queue'
        pass

    # Auxiliary operations
    
    def first(self):
        'Return a reference to the first element in the double-ended queue'
        pass

    def last(self):
        'Return a reference to the last element in the double-ended queue'
        pass

    def is_empty(self):
        pass

    def __len__(self):
        pass

An array-based implementation would implement all operations in **amortized** $O(1)$ time due to the occasional need to resize the array. A linked-list based implementation achieves all operations in worst-case $O(1)$ time! 

In [86]:
class LinkedDeque(DoublyLinkedList):
    def add_first(self, e):
        'Add element e to the front of the double-ended queue'
        self._insert_between(e, self.header, self.header.next)

    def add_last(self, e):
        'Add element e to the end of the double-ended queue'
        self._insert_between(e, self.trailer.prev, self.trailer)

    # Renamed deque operation
    def remove_first(self):
        'Remove the first element from the double-ended queue'
        if self.is_empty():
            raise Empty('Deque is empty')
        return self._delete_node(self.header.next)

    def remove_last(self):
        'Remove the last element from the double-ended queue'
        if self.is_empty():
            raise Empty('Deque is empty')
        return self._delete_node(self.trailer.prev)
    
    # Auxiliary operations
    
    def first(self):
        'Return a reference to the first element in the double-ended queue'
        if self.is_empty():
            raise Empty('Deque is empty')
        return self.header.next.element        

    def last(self):
        'Return a reference to the last element in the double-ended queue'
        if self.is_empty():
            raise Empty('Deque is empty')
        return self.trailer.prev.element

    def is_empty(self):
        return self.size == 0

    def __len__(self):
        return self.size

In [89]:
ld = LinkedDeque()
ld.add_first(1)
ld.add_first(2)
ld.add_last('a')

In [90]:
cursor = ld.header.next
while cursor is not None:
    print(cursor.element)
    cursor = cursor.next

2
1
a
None


### Python's built-in deque

The python standard library provides a deque data structure in the collections module. 
The deque is implemented using a doubly-linked list.

In [56]:
from collections import deque 
    
# Declaring deque 
de = deque(['A','B','C'])  
    
de.append('D')

de 

deque(['A', 'B', 'C', 'D'])

In [58]:
print(de.popleft())
de

A


deque(['B', 'C', 'D'])

In [60]:
print(de.pop())
de

D


deque(['B', 'C'])

In [62]:
print(de.pop())
de

C


deque(['B'])

In [64]:
de.appendleft('E')
de

deque(['E', 'B'])

## Summary

### Advantages of Array-Based Sequences

* $O(1)$ integer-based indexing
* $O(n)$ worst case insertion, but appending at the end is $O(1)$ amortized.
* In many use cases, operations on array-based sequences are faster than link-based sequences (by a constant factor).
* Use proportionally less memory.

### Advantage of Link-Based Sequences

* The $O(1)$ operations are worst-case bounds. No amortization. Including adding/deleting from either end.
* General (anywhere) $O(1)$ insertions and deletions (if a reference to a "current" node is maintained).
